<a href="https://colab.research.google.com/github/ariegever/ImageProcessing_Project/blob/main/1_unet_making_assest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This script prepares your Sentinel-1 and Sentinel-2 data in Earth Engine.
# Run this script first to create the assets you will use in the next script.

import ee
import geemap # This library makes GEE in Python much easier
import json
import os
import config
import utils

print("Authenticating and Initializing Earth Engine...")
# Trigger GEE authentication
ee.Authenticate()
ee.Initialize(project=config.PROJECT_ID)
print("Earth Engine Initialized.")

# --- 1. USER INPUT ---
# Variables are loaded from config.py
print(f"Year: {config.YEAR}")

# --- 2. LOAD SOURCE ASSETS ---
print(f"Loading Land Cover asset: {config.LC_ASSET_ID}")
try:
    lc_image = ee.Image(config.LC_ASSET_ID)
except ee.EEException as e:
    print(f"Error loading LC Asset ID: {e}")
    print("Please double-check your LC_ASSET_ID path in config.py.")
    # Stop the script if the main asset can't be loaded.
    raise

# Get the geometry, CRS, and scale from your LC map
# All new imagery will be matched to this.
geometry = lc_image.geometry()
crs = lc_image.projection().crs()
scale = lc_image.projection().nominalScale()

print(f"LC asset loaded. CRS: {crs.getInfo()}, Scale: {scale.getInfo()}m")
print("Target geometry (bounding box):")
print(geometry.bounds().getInfo()['coordinates'])

# --- 3. PROCESS SENTINEL-2 ---

print("\nProcessing Sentinel-2 imagery...")
# Load S2, filter, mask clouds, and create a median composite
s2_image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(geometry)
            .filterDate(config.START_DATE, config.END_DATE)
            .map(utils.maskS2clouds)
            .median()
            .clip(geometry)
            )

print("Sentinel-2 processing complete.")

# --- 4. PROCESS SENTINEL-1 ---
print("\nProcessing Sentinel-1 imagery...")
# Load S1, filter, and create a median composite
s1_image = (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(geometry)
            .filterDate(config.START_DATE, config.END_DATE)
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
            .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .select(['VV', 'VH'])
            .median()
            .clip(geometry)
           )
# --- 5. EXPORT ASSETS ---
print("\nStarting asset export tasks...")

# Create the asset folder if it doesn't exist
try:
    # Use ee.data.createAsset for folder creation if needed, or rely on geemap if available and working
    # Here we assume the folder might exist or the user handles it.
    # geemap.create_ee_folder(config.ASSET_EXPORT_FOLDER) 
    print(f"Export folder: {config.ASSET_EXPORT_FOLDER}")
except Exception as e:
    print(f"Could not create/verify asset folder: {e}. Assuming it exists.")

# Create and start the export tasks
task_s2 = ee.batch.Export.image.toAsset(
    image=s2_image,
    description=f'Export_{config.AOI_NAME}_Sentinel2_{config.YEAR}',
    assetId=config.S2_ASSET_ID,
    region=geometry,
    scale=scale,
    crs=crs,
    maxPixels=1e13
)
task_s2.start()

task_s1 = ee.batch.Export.image.toAsset(
    image=s1_image,
    description=f'Export_{config.AOI_NAME}_Sentinel1_{config.YEAR}',
    assetId=config.S1_ASSET_ID,
    region=geometry,
    scale=scale,
    crs=crs,
    maxPixels=1e13
)
task_s1.start()

print("\n--- TASKS STARTED ---")
print("Check the 'Tasks' tab in your Google Earth Engine Code Editor.")
print("Wait for the tasks to complete before running the next script.")
print("\nOnce tasks are complete, use these Asset IDs in the next script (automatically loaded from config.py):")
print(f"LC_ASSET_ID: {config.LC_ASSET_ID}")
print(f"S2_ASSET_ID: {config.S2_ASSET_ID}")
print(f"S1_ASSET_ID: {config.S1_ASSET_ID}")
print("\n--- TASKS STARTED ---")